In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import gym
import ptan
from typing import List, Optional, Tuple, Any

/usr/local/lib/python3.11/dist-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
class ToyEnv(gym.Env):
    """
    Środowisko z obserwacjami o wartościach od 0 do 4 i akcjami od 0 do 2.
    Obserwacje zmieniają się cyklicznie zgodnie z operacją modulo 5,
	  a nagroda jest równa wartości akcji.
    Epizody mają stałą długość równą 10.
    """

    def __init__(self):
        super(ToyEnv, self).__init__()
        self.observation_space = gym.spaces.Discrete(n=5)
        self.action_space = gym.spaces.Discrete(n=3)
        self.step_index = 0

    def reset(self):
        self.step_index = 0
        return self.step_index, {}

    def step(self, action):
        is_done = self.step_index == 10
        if is_done:
            # Dodanie is_truncated (zawsze False w tym przypadku)
            return self.step_index % self.observation_space.n, \
                   0.0, is_done, False, {}
        self.step_index += 1
        # Dodanie is_truncated (zawsze False w tym przypadku)
        return self.step_index % self.observation_space.n, \
               float(action), self.step_index == 10, False, {}


class DullAgent(ptan.agent.BaseAgent):
    """
    Agent zawsze zwraca taką samą akcję.
    """
    def __init__(self, action: int):
        self.action = action

    def __call__(self, observations: List[Any],
                 state: Optional[List] = None) \
            -> Tuple[List[int], Optional[List]]:
        return [self.action for _ in observations], state

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Klasa ExperienceSource
# generuje fragmenty trajektorii utworzonych przez akcje agenta w środowisku z wszystkimi krokami pośrednimi
# wykorzystuje kilka argumentów: środowisko, instancję agenta, liczbę kroków
# opcjonalny jest parametr vectorized (domyślnie False)
# jeżeli True, środowisko musi być wektoryzowanym środowiskiem OpenAI Universe

env = ToyEnv()
agent = DullAgent(action=1)
exp_source = ptan.experience.ExperienceSource(env, agent, steps_count=2)
for idx, exp in enumerate(exp_source): # każda iteracja zwraca fragment trajektorii agenta
# w postaci krotek o długości równej parametrowi step_count lub mniejszej
    if idx > 2:
        break
    print(exp)

# Co dzieje się w trakcie iteracji:
# 1. wywoływana jest funkcja reset() dla środowiska - uzyskanie stanu początkowego
# 2. agent wybiera akcję na podstawie zwróconego stanu
# 3. wywoływana jest metoda step() dla uzyskania nagrody i następnego stanu
# 4. następny stan jest przekazywany agentowi w celu wykonania kolejnej akcji
# 5. zwrócona zostaje informacja o przejściu z jednego stanu do drugiego
# 6. kroki 3. - 5. zostają powtórzone, aż do przetworzenia źródła doświadczenia

(Experience(state=0, action=1, reward=1.0, done_trunc=False), Experience(state=1, action=1, reward=1.0, done_trunc=False))
(Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False))
(Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False))


In [4]:
# Każdy obiekt w krotce zwracanej przez ExperienceSource jest instancją klasy ptan.experience.Experience
# ta klasa to kontener namedtuple z polami:
# 1. state - stan obserwowany przed podjęciem akcji
# 2. action - wykonana akcja
# 3. reward - natychmiastowa nagroda otrzymana ze środowiska
# 4. done_trunc - flaga oznaczająca zakończenie epizodu

# jeżeli epizod dobiegnie końca, fragment trajektorii będzie krótszy, a środowisko zostanie automatycznie zresetowane

for idx, exp in enumerate(exp_source):
    if idx > 15:
        break
    print(exp)

(Experience(state=0, action=1, reward=1.0, done_trunc=False), Experience(state=1, action=1, reward=1.0, done_trunc=False))
(Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False))
(Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False))
(Experience(state=3, action=1, reward=1.0, done_trunc=False), Experience(state=4, action=1, reward=1.0, done_trunc=False))
(Experience(state=4, action=1, reward=1.0, done_trunc=False), Experience(state=0, action=1, reward=1.0, done_trunc=False))
(Experience(state=0, action=1, reward=1.0, done_trunc=False), Experience(state=1, action=1, reward=1.0, done_trunc=False))
(Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False))
(Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False))
(Experience(stat

In [5]:
# możemy zażądać od klasy ExperienceSource wygenerowania trajektorii składowych o dowolnej długości

exp_source = ptan.experience.ExperienceSource(
        env=env, agent=agent, steps_count=4)
print(next(iter(exp_source)))

(Experience(state=0, action=1, reward=1.0, done_trunc=False), Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False))


In [6]:
# obiektowi ww. klasy można przekazać kilka instancji klasy gym.Env - używane będą cyklicznie

env1 = ToyEnv()
env2 = ToyEnv()

exp_source = ptan.experience.ExperienceSource(
        env=[env1, env2], agent=agent, steps_count=4)
for idx, exp in enumerate(exp_source):
    if idx > 4:
        break
    print(exp)

(Experience(state=0, action=1, reward=1.0, done_trunc=False), Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False))
(Experience(state=0, action=1, reward=1.0, done_trunc=False), Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False))
(Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False), Experience(state=4, action=1, reward=1.0, done_trunc=False))
(Experience(state=1, action=1, reward=1.0, done_trunc=False), Experience(state=2, action=1, reward=1.0, done_trunc=False), Experience(state=3, action=1, reward=1.0, done_trunc=False), Experience(state=4, action=1, reward=1.0, done_trunc=False))
(Experience(state=2,

In [7]:
# klasa ExperienceSourceFirstLast
# zamiast pełnej trajektorii składowej zwraca jedynie krok pierwszy i krok ostatni oraz dba o właściwą akumulację nagrody
# dziedziczy po ExperienceSource, ale zwraca inne dane - krotkę z polami:
# 1. state - stan, w którym zdecydowano się na podjęcie akcji
# 2. action - akcja wykonana w danym kroku
# 3. reward - częściowo skumulowana nagroda za step_count kroków (jeżeli step_count=1, nagroda natychmiastowa)
# 4. last_state - stan osiągnięty po wykonaniu akcji (jeżeli epizod się kończy, wartością jest None)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=1.0, steps_count=1)
for idx, exp in enumerate(exp_source):
    if idx > 10:
        break
    print(exp)

ExperienceFirstLast(state=0, action=1, reward=1.0, last_state=1)
ExperienceFirstLast(state=1, action=1, reward=1.0, last_state=2)
ExperienceFirstLast(state=2, action=1, reward=1.0, last_state=3)
ExperienceFirstLast(state=3, action=1, reward=1.0, last_state=4)
ExperienceFirstLast(state=4, action=1, reward=1.0, last_state=0)
ExperienceFirstLast(state=0, action=1, reward=1.0, last_state=1)
ExperienceFirstLast(state=1, action=1, reward=1.0, last_state=2)
ExperienceFirstLast(state=2, action=1, reward=1.0, last_state=3)
ExperienceFirstLast(state=3, action=1, reward=1.0, last_state=4)
ExperienceFirstLast(state=4, action=1, reward=1.0, last_state=None)
ExperienceFirstLast(state=0, action=1, reward=1.0, last_state=1)


In [8]:
# dane te są lepsze w przypadku treningu głębokiej sieci Q, bo można ich użyć w przybliżeniu Bellmana bezpśrednio
# sprawdźmy wynik w razie zastosowania większej liczby kroków

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=1.0, steps_count=2)
for idx, exp in enumerate(exp_source):
    if idx > 10:
        break
    print(exp)

ExperienceFirstLast(state=0, action=1, reward=2.0, last_state=2)
ExperienceFirstLast(state=1, action=1, reward=2.0, last_state=3)
ExperienceFirstLast(state=2, action=1, reward=2.0, last_state=4)
ExperienceFirstLast(state=3, action=1, reward=2.0, last_state=0)
ExperienceFirstLast(state=4, action=1, reward=2.0, last_state=1)
ExperienceFirstLast(state=0, action=1, reward=2.0, last_state=2)
ExperienceFirstLast(state=1, action=1, reward=2.0, last_state=3)
ExperienceFirstLast(state=2, action=1, reward=2.0, last_state=4)
ExperienceFirstLast(state=3, action=1, reward=2.0, last_state=None)
ExperienceFirstLast(state=4, action=1, reward=1.0, last_state=None)
ExperienceFirstLast(state=0, action=1, reward=2.0, last_state=2)
